In [2]:
!pip install gensim

  Using cached gensim-4.3.3-cp310-cp310-win_amd64.whl.metadata (8.2 kB)
Using cached gensim-4.3.3-cp310-cp310-win_amd64.whl (24.0 MB)


In [ ]:
#⚙️ Installing Required Libraries
%pip install gensim


# 📦 Import Libraries
 
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
import numpy as np
from tqdm import tqdm

# 📌 Make sure to download NLTK data:

nltk.download('punkt')
nltk.download('wordnet')

# ✏️ Preprocessing the Text
# ✔️ Use Lemmatization instead of Stemming
# ❌ Don’t remove stop words (to test vector generation for every word)

 
lemmatizer = WordNetLemmatizer()
corpus = []

# Iterate over the actual message texts, not the DataFrame itself
for msg in messages['message']:
    msg = re.sub('[^a-zA-Z]', ' ', msg)
    msg = msg.lower()
    msg = msg.split()
    msg = [lemmatizer.lemmatize(word) for word in msg]
    corpus.append(' '.join(msg))

print(corpus[:2])  # Show first 2 preprocessed messages

# 🧹 Tokenize Sentences and Words Using simple_preprocess
# 📌 simple_preprocess() converts text to lowercase, removes short/long words

# 🔄 Loop over sentences and tokenize them

 
words = []
for sentence in corpus:
    for sent in sent_tokenize(sentence):
        words.append(simple_preprocess(sent))
print(words) # [['label'], ['message']]

# 🧠 Train Word2Vec from Scratch
 
model = Word2Vec(
    sentences=words,
    vector_size=100,  # each word becomes a 100-dimensional vector
    window=5,
    min_count=1
)
#gives all the words (vocabulary)
print(model.wv.key_to_index) # {'label': 0, 'message': 1}
# gives the index of each word in the vocabulary
print(model.wv.index_to_key) # ['label', 'message']

# 📌 Check vocabulary size:
print(model.corpus_count)
# 2


# 📌 Vector for word “good”:
# Print vector for a word that exists in the vocabulary, e.g., 'go'
print(model.wv['go'])  # vector for 'go'
# Example output: array([...])

# 🧮 Average Word2Vec Function
def avg_word2vec(doc, model):
    # Get all word vectors for words in vocab
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        # Return a zero vector if no known words
        return np.zeros(model.vector_size)

# ⚙️ Convert All Sentences to Vectors
X = []

for sentence in tqdm(words):
    X.append(avg_word2vec(sentence, model))

# 🔎 Now X is a list of 100-dimensional sentence vectors
X_new = np.vstack(X)
print(X_new.shape)  # Output: (number_of_sentences, 100)

# 🔢 First sentence vector:

print(X_new[0])
print(X_new[0].shape)  # Output: (100,)

# 🎯 Target Variable (Output / Labels)
# 📌 Assume y contains the output labels like 1 for spam and 0 for ham

# Example dataset
y = messages['label']  # Example: 'spam' or 'ham'
# Align y with X_new
y = y.iloc[:len(X_new)].reset_index(drop=True)
y
# 0        ham
# 1        ham
# 2       spam
# 3        ham
# 4        ham
#         ... 
# 5567    spam
# 5568     ham
# 5569     ham
# 5570     ham
# 5571     ham
# Name: label, Length: 5572, dtype: object

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 🧪 Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# 🤖 Train Logistic Regression
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

# 📈 Predict
y_pred = model_lr.predict(X_test)

# ✅ Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))
# Accuracy: 0.8608617594254937

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sahus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sahus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat', 'ok lar joking wif u oni']
[['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'great', 'world', 'la', 'buffet', 'cine', 'there', 'got', 'amore', 'wat'], ['ok', 'lar', 'joking', 'wif', 'oni'], ['free', 'entry', 'in', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', 'st', 'may', 'text', 'fa', 'to', 'to', 'receive', 'entry', 'question', 'std', 'txt', 'rate', 'apply', 'over'], ['dun', 'say', 'so', 'early', 'hor', 'already', 'then', 'say'], ['nah', 'don', 'think', 'he', 'go', 'to', 'usf', 'he', 'life', 'around', 'here', 'though'], ['freemsg', 'hey', 'there', 'darling', 'it', 'been', 'week', 'now', 'and', 'no', 'word', 'back', 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', 'tb', 'ok', 'xxx', 'std', 'chgs', 'to', 'send', 'to', 'rcv'], ['even', 'my', 'brother', 'is', 'not', 'like', 'to', 'speak', 'with', 'me', 'they', 'treat', 'me', 'like

100%|██████████| 5569/5569 [00:01<00:00, 5257.67it/s]

(5569, 100)
[-0.15430196  0.26905397  0.14129858  0.05302018  0.14662583 -0.64586079
  0.21779943  0.64853978 -0.23578474 -0.13678972 -0.2249057  -0.56048787
 -0.11315049  0.11277078  0.13334414 -0.19710308 -0.00232021 -0.43405256
  0.00383473 -0.60233235  0.39621189  0.17002256  0.27098566 -0.2017846
 -0.03338306  0.11456402 -0.26844823 -0.12848336 -0.27415302 -0.00979146
  0.34254381 -0.0135604   0.14537956 -0.36166683 -0.09539386  0.32493353
 -0.00500245 -0.2001639  -0.23147948 -0.4755463   0.11162233 -0.22452582
 -0.37382326  0.10077342  0.18028516 -0.094125   -0.20822495 -0.08312199
  0.14331204  0.21980548  0.22133939 -0.30393863 -0.1081342  -0.08570249
 -0.2050605   0.1183987   0.1375836  -0.05024294 -0.35131371  0.15802689
  0.07201726  0.01267604 -0.0037381  -0.02053454 -0.25956768  0.36930576
  0.12347971  0.3593353  -0.47746897  0.44457749 -0.3069551   0.23310536
  0.31084874 -0.17580107  0.41486934  0.09742448 -0.11419889 -0.04557159
 -0.10784844  0.02171759 -0.24834315 -0.